In [20]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [21]:
num_qubits = 2
max_weight = 1
term_dict = {("S", "XX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [22]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    intersect = used_indices_1.intersection(used_indices_2)
    if len(intersect) > 0 and len(intersect) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
prep_string_iterator = product([1,-1],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))
prep_meas_pair

[(stim.PauliString("+XX"), stim.PauliString("+XX")),
 (stim.PauliString("+XX"), stim.PauliString("+XY")),
 (stim.PauliString("+XX"), stim.PauliString("+XZ")),
 (stim.PauliString("+XX"), stim.PauliString("+YX")),
 (stim.PauliString("+XX"), stim.PauliString("+YY")),
 (stim.PauliString("+XX"), stim.PauliString("+YZ")),
 (stim.PauliString("+XX"), stim.PauliString("+ZX")),
 (stim.PauliString("+XX"), stim.PauliString("+ZY")),
 (stim.PauliString("+XX"), stim.PauliString("+ZZ")),
 (stim.PauliString("+XY"), stim.PauliString("+XX")),
 (stim.PauliString("+XY"), stim.PauliString("+XY")),
 (stim.PauliString("+XY"), stim.PauliString("+XZ")),
 (stim.PauliString("+XY"), stim.PauliString("+YX")),
 (stim.PauliString("+XY"), stim.PauliString("+YY")),
 (stim.PauliString("+XY"), stim.PauliString("+YZ")),
 (stim.PauliString("+XY"), stim.PauliString("+ZX")),
 (stim.PauliString("+XY"), stim.PauliString("+ZY")),
 (stim.PauliString("+XY"), stim.PauliString("+ZZ")),
 (stim.PauliString("+XZ"), stim.PauliString("+

In [23]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        prep_string_attributes,
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        prep_string_attributes,
        measure_string_attributes,
    )
)



import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[1:] for s in el[-4:-1])
            egen = ",".join(str(s) for s in el[:-4])
            coef = int(el[-1])
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])


ValueError: Length of values (486) does not match length of index (243)

In [24]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [26]:
def pauli_pairs_to_circuits(pauli_pair, pauli_basis_dict):
    sgn = pauli_pair[0].sign
    circuits = []
    for pauli_str in pauli_pair:
        opstr = []
        for i, term in enumerate(str(pauli_str)[1:]):
            if i == 0:
                key = sgn + term
            else:
                key = "+" + term
            opstr.extend([Label(opname, i) for opname in pauli_basis_dict[key]])
        circuits.append(Circuit(opstr, num_lines=len(pauli_str)))
    return circuits

In [ ]:
for pair in prep_meas_pair:
    print(pauli_pairs_to_circuits(pair, pauli_basis_dict))

In [6]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gx", "Gy", "Gcnot"]
max_lengths = [1, 2]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in range(num_qubits))]},
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in range(num_qubits)))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, maxweight=1, idle_string=global_idle_string
    )
idle_experiments

[Circuit(Gy:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gy:0Gi:0Gx:0@(0)),
 Circuit(Gy:0Gi:0Gi:0Gx:0@(0)),
 Circuit(Gy:0Gi:0@(0)),
 Circuit(Gy:0Gi:0Gi:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gx:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gi:0Gx:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0@(0)),
 Circuit(Gx:0Gx:0Gx:0Gi:0Gi:0@(0)),
 Circuit(Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gi:0Gi:0Gy:0Gy:0Gy:0@(0)),
 Circuit(Gi:0Gx:0@(0)),
 Circuit(Gi:0Gi:0Gx:0@(0)),
 Circuit(Gi:0@(0)),
 Circuit(Gi:0Gi:0@(0))]

In [7]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(
pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")
# updated_ckt_list = []


# for ckt in idle_experiments:
#     new_ckt = ckt.copy(editable=True)
#     for i, lbl in enumerate(ckt):
#         if lbl == Label(()):
#             # new_ckt[i] = [Label(("Gi", i)) for i in range(n_qubits)]
#             new_ckt[i] = Label("Gi", tuple(i for i in range(num_qubits)))
#             # new_ckt[i] = Label(("Gi", 0))
#     updated_ckt_list.append(new_ckt)

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    mdl_datagen, idle_experiments, 10000000, seed=8675309, sample_error="none"
)
ds

KeyError: 'XX'

In [ ]:
print(mdl_datagen.operation_blks["gates"]["Gi"].factorops[1].errorgen_coefficients())

OrderedDict([(('S', ('XX',), (0, 1)), 0.0001)])


In [ ]:
print(mdl_datagen)

layers:rho0 = Computational Z-basis state vec for 2 qubits w/z-values: [0 0]

layers:Mdefault = Computational(Z)-basis POVM on 2 qubits and filter None


gates:Gi = 
Composed operation of 2 factors:
Factor 0:
StaticUnitaryOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0 1.00   0
   0   0   0 1.00
Factor 1:
Exponentiated operation map with dim = 16, num params = 1


gates:Gx = 
StaticStandardOp with name Gx and evotype densitymx


gates:Gy = 
StaticStandardOp with name Gy and evotype densitymx


gates:Gcnot = 
StaticStandardOp with name Gcnot and evotype densitymx


layers:Gi:0:1 = 
Composed operation of 2 factors:
Factor 0:
StaticUnitaryOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0 1.00   0
   0   0   0 1.00
Factor 1:
Exponentiated operation map with dim = 16, num params = 1


layers:Gx:0 = 
Embedded operation with full dimension 16 and state space QubitSpace((0, 1))
 that embeds the following 4-dimensional operation into acting on the (0,) space
St

In [ ]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    # print(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_observables = all_full_length_observables(
            pauli_fidpair[1], nqubits
        )
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
            )

            #info["jacobian row"] = full_jacobian[ifp]
            infos_for_this_fidpair[out] = info
            # ic(infos_for_this_fidpair)
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("+","").replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
            # obs_err_rates = np.concatenate([np.array([
            #                 observed_error_rates[i]
            #                 for i in range(len(all_fidpairs))
            #                 ]
            #             )
            #         ]
            #     )
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [ ]:
idl_string = "Gi:"+":".join((str(i) for i in range(num_qubits)))
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts)

In [ ]:
observed_error_rates

{0: [4.965068306494546e-17, 4.965068306494546e-17, 4.965068306494546e-17],
 1: [4.965068306494546e-17, 0.0, 0.0],
 2: [4.965068306494546e-17, 0.0, 0.0],
 3: [0.0, 4.965068306494546e-17, 0.0],
 4: [0.0, 0.0, 0.0],
 5: [0.0, 0.0, 0.0],
 6: [0.0, 4.965068306494546e-17, 0.0],
 7: [0.0, 0.0, 0.0],
 8: [0.0, 0.0, 0.0],
 9: [4.965068306494546e-17, 0.0, 0.0],
 10: [4.965068306494546e-17, -0.00019994002864205597, -0.00019994002864205597],
 11: [4.965068306494546e-17, 0.0, 0.0],
 12: [0.0, 0.0, 0.0],
 13: [0.0, -0.00019994002864205597, 0.0],
 14: [0.0, 0.0, 0.0],
 15: [0.0, 0.0, 0.0],
 16: [0.0, -0.00019994002864205597, 0.0],
 17: [0.0, 0.0, 0.0],
 18: [4.965068306494546e-17, 0.0, 0.0],
 19: [4.965068306494546e-17, 0.0, 0.0],
 20: [4.965068306494546e-17, -0.00019994002864205597, -0.00019994002864205597],
 21: [0.0, 0.0, 0.0],
 22: [0.0, 0.0, 0.0],
 23: [0.0, -0.00019994002864205597, 0.0],
 24: [0.0, 0.0, 0.0],
 25: [0.0, 0.0, 0.0],
 26: [0.0, -0.00019994002864205597, 0.0],
 27: [0.0, 4.965068306

In [ ]:
obs_error_rates_by_exp

{('XX', 'XX', ' X_'): 4.965068306494546e-17,
 ('XX', 'XX', ' _X'): 4.965068306494546e-17,
 ('XX', 'XX', ' XX'): 4.965068306494546e-17,
 ('XX', 'XY', ' X_'): 4.965068306494546e-17,
 ('XX', 'XY', ' _Y'): 0.0,
 ('XX', 'XY', ' XY'): 0.0,
 ('XX', 'XZ', ' X_'): 4.965068306494546e-17,
 ('XX', 'XZ', ' _Z'): 0.0,
 ('XX', 'XZ', ' XZ'): 0.0,
 ('XX', 'YX', ' Y_'): 0.0,
 ('XX', 'YX', ' _X'): 4.965068306494546e-17,
 ('XX', 'YX', ' YX'): 0.0,
 ('XX', 'YY', ' Y_'): 0.0,
 ('XX', 'YY', ' _Y'): 0.0,
 ('XX', 'YY', ' YY'): 0.0,
 ('XX', 'YZ', ' Y_'): 0.0,
 ('XX', 'YZ', ' _Z'): 0.0,
 ('XX', 'YZ', ' YZ'): 0.0,
 ('XX', 'ZX', ' Z_'): 0.0,
 ('XX', 'ZX', ' _X'): 4.965068306494546e-17,
 ('XX', 'ZX', ' ZX'): 0.0,
 ('XX', 'ZY', ' Z_'): 0.0,
 ('XX', 'ZY', ' _Y'): 0.0,
 ('XX', 'ZY', ' ZY'): 0.0,
 ('XX', 'ZZ', ' Z_'): 0.0,
 ('XX', 'ZZ', ' _Z'): 0.0,
 ('XX', 'ZZ', ' ZZ'): 0.0,
 ('XY', 'XX', ' X_'): 4.965068306494546e-17,
 ('XY', 'XX', ' _X'): 0.0,
 ('XY', 'XX', ' XX'): 0.0,
 ('XY', 'XY', ' X_'): 4.965068306494546e-17,
 

In [ ]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [ ]:
j = df.to_numpy()

In [ ]:
jinv = np.linalg.pinv(j)

In [ ]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [ ]:
dict(zip(df.columns,intrins_errs))

{'H,+X_': 2.541098841762901e-21,
 'H,+Y_': -4.0657581468206416e-20,
 'H,+Z_': -3.0627089976185504e-20,
 'H,+_X': -1.9905274260476058e-20,
 'H,+_Y': -6.3527471044072525e-21,
 'H,+_Z': 5.382399448946647e-20,
 'H,+XX': -1.9905274260476058e-20,
 'H,+XY': 3.303361328028375e-20,
 'H,+XZ': -2.964615315390051e-20,
 'H,+YX': 3.653630260602412e-20,
 'H,+YY': 2.371692252312041e-20,
 'H,+YZ': -4.277516383634217e-20,
 'H,+ZX': 1.334076891925523e-20,
 'H,+ZY': 5.95040645446146e-20,
 'H,+ZZ': -8.175318042876784e-21,
 'S,+X_': -2.3399285167900047e-18,
 'S,+Y_': -2.3469165386048527e-18,
 'S,+Z_': -2.3435284068158355e-18,
 'S,+_X': -2.4275964268308248e-18,
 'S,+_Y': -2.3458577474207848e-18,
 'S,+_Z': -2.4172202732269596e-18,
 'S,+XX': 9.997001432104963e-05,
 'S,+XY': -3.0560948736935156e-18,
 'S,+XZ': -3.0298368523286323e-18,
 'S,+YX': -2.9849441061241544e-18,
 'S,+YY': -3.1179282788430795e-18,
 'S,+YZ': -3.1433392672607086e-18,
 'S,+ZX': -3.1721383874673548e-18,
 'S,+ZY': -3.1458803661024715e-18,
 'S,+